In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'bankdata:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5095892%2F8532117%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240527%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240527T151529Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D61802faeb67c208eb065a30db06270b6564344012d2c60fc7bb8eb2934107391a7169b66ddd80d1c3ba46838dcb97aebd93ba289ec25c50375d5d31bb4e0c7b64b49c0c6aecf88ccb76fb06d045656b563a23fcb692268266207324050563e79da2ab7916db1c4b66953388053d4979db6941c74426ff0bf8a829cad17995af04d8aa69af8d1de62a3a8721b32fcdda26e616045575bc8eb046229b35240977bd6ab9fb4c8dfc56c0da6c0c79000ac0e873d0b98e41ef371fd54a9c9bbbf27462979082d52516f8917a9f5e4090386e45c83db0ef3df8adb69450636a76535fdb6bb15354a142d5cff3ca562c91a99023c8fe2fc0c661844aafff5b9f6549672'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bankdata/bank-additional-full.csv


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset
file_path = '/kaggle/input/bankdata/bank-additional-full.csv'
data = pd.read_csv(file_path)

# Check for missing values
print(data.isnull().sum())


age;"job";"marital";"education";"default";"housing";"loan";"contact";"month";"day_of_week";"duration";"campaign";"pdays";"previous";"poutcome";"emp.var.rate";"cons.price.idx";"cons.conf.idx";"euribor3m";"nr.employed";"y"    0
dtype: int64


In [ ]:
# Perform one-hot encoding for categorical variables
data_encoded = pd.get_dummies(data, drop_first=True)

# Obtain the column names after one-hot encoding
column_names = data_encoded.columns

print(column_names)

Index(['age;"job";"marital";"education";"default";"housing";"loan";"contact";"month";"day_of_week";"duration";"campaign";"pdays";"previous";"poutcome";"emp.var.rate";"cons.price.idx";"cons.conf.idx";"euribor3m";"nr.employed";"y"_17;"student";"single";"basic.9y";"no";"yes";"no";"cellular";"aug";"fri";182;2;999;2;"failure";-2.9;92.201;-31.4;0.869;5076.2;"no"',
       'age;"job";"marital";"education";"default";"housing";"loan";"contact";"month";"day_of_week";"duration";"campaign";"pdays";"previous";"poutcome";"emp.var.rate";"cons.price.idx";"cons.conf.idx";"euribor3m";"nr.employed";"y"_17;"student";"single";"basic.9y";"no";"yes";"no";"cellular";"aug";"fri";92;3;4;2;"success";-2.9;92.201;-31.4;0.869;5076.2;"no"',
       'age;"job";"marital";"education";"default";"housing";"loan";"contact";"month";"day_of_week";"duration";"campaign";"pdays";"previous";"poutcome";"emp.var.rate";"cons.price.idx";"cons.conf.idx";"euribor3m";"nr.employed";"y"_17;"student";"single";"unknown";"no";"no";"yes";"cel

In [ ]:
# Split each column name by semicolon and extract the last element as the actual column name
column_names = [col.split(';')[-1] for col in column_names]

# Remove any leading or trailing whitespaces from column names
column_names = [col.strip() for col in column_names]


In [ ]:
print(data_encoded.columns)


Index(['age;"job";"marital";"education";"default";"housing";"loan";"contact";"month";"day_of_week";"duration";"campaign";"pdays";"previous";"poutcome";"emp.var.rate";"cons.price.idx";"cons.conf.idx";"euribor3m";"nr.employed";"y"_17;"student";"single";"basic.9y";"no";"yes";"no";"cellular";"aug";"fri";182;2;999;2;"failure";-2.9;92.201;-31.4;0.869;5076.2;"no"',
       'age;"job";"marital";"education";"default";"housing";"loan";"contact";"month";"day_of_week";"duration";"campaign";"pdays";"previous";"poutcome";"emp.var.rate";"cons.price.idx";"cons.conf.idx";"euribor3m";"nr.employed";"y"_17;"student";"single";"basic.9y";"no";"yes";"no";"cellular";"aug";"fri";92;3;4;2;"success";-2.9;92.201;-31.4;0.869;5076.2;"no"',
       'age;"job";"marital";"education";"default";"housing";"loan";"contact";"month";"day_of_week";"duration";"campaign";"pdays";"previous";"poutcome";"emp.var.rate";"cons.price.idx";"cons.conf.idx";"euribor3m";"nr.employed";"y"_17;"student";"single";"unknown";"no";"no";"yes";"cel

In [ ]:
# Assuming the target variable name is 'y'
data_encoded = data_encoded.rename(columns={data_encoded.columns[0]: 'y'})

# Define the feature matrix X (excluding the target variable column)
X = data_encoded.drop('y', axis=1)

# Define the target vector y
y = data_encoded['y']


In [ ]:

# Assuming 'data_encoded' is your DataFrame containing the dataset
# Extract the target variable column
y = data_encoded['y']

# Ensure the target variable is encoded as 0s and 1s (if not already)
# For example, if 'yes' and 'no' are the labels, we can encode 'yes' as 1 and 'no' as 0
y_binary = y.map({'yes': 1, 'no': 0})

# Convert the target variable column into a 1-dimensional array
y_train = y_binary.values

# Verify the shape of y_train
print(y_train.shape)


(41188,)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Split the data into features (X) and target variable (y)
X = data_encoded.drop('y', axis=1)
y = data_encoded['y']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the logistic regression model
model = LogisticRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Example: Predict on the test set
y_pred = model.predict(X_test)

# Example: Evaluate the model
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)


Accuracy: 1.0
